In [1]:
import os
import numpy as np
import pandas as pd
import gc

from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn import decomposition, preprocessing

from functions import *
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# debug
debug = False
if not debug:
    num_rows = None
else:
    num_rows = 10000
    
def str_to_datetime(strg):
    return datetime.strptime(strg, '%Y-%m-%d %H:%M:%S')
    

# Présentation du jeu de données

**Ce diagramme présente les relations entre les données:**

![image](https://i.imgur.com/HRhd2Y0.png)

L'objectif est de segmenter les clients. Cependant, la base de donnée "customers" contient non pas une ligne par client mais une ligne par commande. On va donc créer notre propre dataframe a partir du jeu de données fourni.

# Analyse exploratoire & Feature engineering

On traite d'abord le fichier client qui associe les IDs client uniques et non uniques. On génère une colonne "nombre de commandes", et une table d'association pour les IDs.

In [2]:
app_customers = pd.read_csv('./data/olist_customers_dataset.csv', nrows=num_rows)
app_aggs = {'customer_id': ['count']}
app = app_customers.groupby('customer_unique_id').agg({**app_aggs})
app.columns = pd.Index(["orders_count"])

# make hash table: customers ids => customer unique ids
cid_to_ucid = pd.DataFrame(data=app_customers[['customer_id', 'customer_unique_id']])
cid_to_ucid = cid_to_ucid.set_index('customer_id')



On traite le fichier des commandes. On ne conserve que les commandes délivrées avec succès (les autres représentant des cas particulier et non des tendances de consommation). Les variables extraites sont le temps de livraison, le delai de paiement, et l'erreur de prediction pour la date de livraison. On génère également une table d'association entre l'ID des commandes et l'ID unique des clients. 

In [3]:
app_orders = pd.read_csv('./data/olist_orders_dataset.csv', nrows=num_rows)
app_orders = app_orders.join(cid_to_ucid, how='left', on='customer_id')
# only keep delivered orders with non NaN timestamps
app_orders = app_orders[app_orders["order_status"]=="delivered"]
app_orders = app_orders[app_orders["order_delivered_customer_date"].isnull()==False]
app_orders = app_orders[app_orders["order_approved_at"].isnull()==False]
app_orders = app_orders[app_orders["order_purchase_timestamp"].isnull()==False]
app_orders = app_orders[app_orders["order_estimated_delivery_date"].isnull()==False]
# convert timestamps to datetime
app_orders["order_delivered_customer_date"] = app_orders["order_delivered_customer_date"].apply(str_to_datetime)
app_orders["order_approved_at"] = app_orders["order_approved_at"].apply(str_to_datetime)
app_orders["order_purchase_timestamp"] = app_orders["order_purchase_timestamp"].apply(str_to_datetime)
app_orders["order_estimated_delivery_date"] = app_orders["order_estimated_delivery_date"].apply(str_to_datetime)
# create new features
app_orders["shipping_delay"] = app_orders["order_delivered_customer_date"] - app_orders["order_approved_at"]
app_orders["payment_delay"] = app_orders["order_approved_at"] - app_orders["order_purchase_timestamp"]
app_orders["delivery_estimation_error"] = app_orders["order_delivered_customer_date"] - app_orders["order_estimated_delivery_date"]
# convert new features to float
app_orders["shipping_delay"] = app_orders["shipping_delay"].apply(lambda x: x.total_seconds() / 86400) # days
app_orders["delivery_estimation_error"] = app_orders["delivery_estimation_error"].apply(lambda x: x.total_seconds() / 86400) # days
app_orders["payment_delay"] = app_orders["payment_delay"].apply(lambda x: x.total_seconds() / 60) # minutes
# add aggregation to main dataframe
orders_aggs = {
    'shipping_delay': ['mean'],
    'payment_delay': ['mean'],
    'delivery_estimation_error': ['mean'],
}
orders = app_orders.groupby('customer_unique_id').agg({**orders_aggs})
orders.columns = pd.Index([e[0] + "_" + e[1] for e in orders.columns.tolist()])
app = app.join(orders, how='left', on='customer_unique_id')
app = app.dropna()

# make hash table: order ids => customer unique ids
oid_to_cuid = pd.DataFrame(data=app_orders[['order_id', 'customer_unique_id']])
oid_to_cuid = oid_to_cuid.set_index('order_id')

On traite le fichier des notes et commentaires pour les commandes délivrées avec succès. On extrait les statistiques associées aux notes, la présence ou non de commentaire et le délai de leur publication.

In [4]:
app_reviews = pd.read_csv('./data/olist_order_reviews_dataset.csv', nrows=num_rows)
app_reviews = app_reviews.join(oid_to_cuid, how='left', on='order_id')
# only keep reviews corresponding to existing orders
app_reviews = app_reviews[app_reviews["order_id"].isin(app_orders["order_id"])]
# convert timestamps to datetime
app_reviews["review_creation_date"] = app_reviews["review_creation_date"].apply(str_to_datetime)
app_reviews["review_answer_timestamp"] = app_reviews["review_answer_timestamp"].apply(str_to_datetime)
# create new features
app_reviews['customer_left_message'] = 1 - app_reviews['review_comment_message'].isnull()
app_reviews['review_delay'] = app_reviews["review_answer_timestamp"] - app_reviews["review_creation_date"]
# convert to float
app_reviews["review_delay"] = app_reviews["review_delay"].apply(lambda x: x.total_seconds() / 86400) # days
# add aggregation to main dataframe
reviews_aggs = {
    'review_score': ['min', 'max', 'mean', 'count'],
    'customer_left_message': ['mean'],
    'review_delay': ['mean']
}
reviews = app_reviews.groupby('customer_unique_id').agg({**reviews_aggs})
reviews.columns = pd.Index([e[0] + "_" + e[1] for e in reviews.columns.tolist()])
app = app.join(reviews, how='left', on='customer_unique_id')

On traite le fichier des paiements. On extrait les types de paiements utilisés après avoir utilisé le one-hot encoding sur la colonne "payment_type", ainsi que le nombre total de mensualités et la valeur totale des paiements.

In [5]:
app_pay = pd.read_csv('./data/olist_order_payments_dataset.csv', nrows=num_rows)
app_pay = app_pay.join(oid_to_cuid, how='left', on='order_id')
# only keep payments corresponding to existing orders
app_pay = app_pay[app_pay["order_id"].isin(app_orders["order_id"])]
# one-hot encoding
app_pay = pd.get_dummies(app_pay, prefix=['payment_type'], columns=['payment_type'])
# add aggregation to main dataframe
pay_aggs = {
    'payment_type_credit_card': ['max'],
    'payment_type_boleto': ['max'],
    'payment_type_debit_card': ['max'],
    'payment_type_voucher': ['max'],
    'payment_installments': ['sum'],
    'payment_value': ['sum']
}
payments = app_pay.groupby('customer_unique_id').agg({**pay_aggs})
payments.columns = pd.Index([e[0] + "_" + e[1] for e in payments.columns.tolist()])
app = app.join(payments, how='left', on='customer_unique_id')

app.head()

,orders_count,shipping_delay_mean,payment_delay_mean,delivery_estimation_error_mean,review_score_min,review_score_max,review_score_mean,review_score_count,customer_left_message_mean,review_delay_mean,payment_type_credit_card_max,payment_type_boleto_max,payment_type_debit_card_max,payment_type_voucher_max,payment_installments_sum,payment_value_sum
customer_unique_id,,,,,,,,,,,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,1,6.400914,14.850000,-4.132905,5,5,5.0,1,1.0,4.572778,1.0,0.0,0.0,0.0,8.0,141.90
0000b849f77a49e4a4ce2b2a4ca5be3f,1,2.984005,434.283333,-4.248125,4,4,4.0,1,0.0,0.937095,1.0,0.0,0.0,0.0,1.0,27.19
0000f46a3911fa3c0805444483337064,1,25.731759,0.000000,-1.389734,3,3,3.0,1,0.0,1.055370,1.0,0.0,0.0,0.0,8.0,86.22
0000f6ccb0745a6a4b88665a16c9f078,1,20.023472,19.600000,-11.108970,4,4,4.0,1,1.0,1.012396,1.0,0.0,0.0,0.0,4.0,43.62
0004aac84e0df4da2b147fca70cf8255,1,13.126435,21.166667,-7.035463,5,5,5.0,1,0.0,4.937211,1.0,0.0,0.0,0.0,6.0,196.89


On traite les fichiers relatifs aux produits et aux commandes. On extrait les valeurs moyennes et écart-types associés de la taille de la description, du nombre de photos, du poids, du volume, du prix et du montant des frais de ports de chaque produit pour chaque client. On réalise également un one-hot encoding sur les catégories (catégories générales nouvellement créées).

In [6]:
def replace_cat(str_cat):
    for main_cat in main_categories:
        if str_cat in main_categories[main_cat]:
            return main_cat
    return 'unknown'

In [7]:
app_products = pd.read_csv('./data/olist_products_dataset.csv', nrows=num_rows)
# translate categories
app_translate = pd.read_csv('./data/product_category_name_translation.csv', nrows=num_rows)
app_products = app_products.set_index('product_category_name')
app_translate = app_translate.set_index('product_category_name')
app_products = app_products.join(app_translate, how='left', on='product_category_name')
app_products = app_products.set_index('product_id') # effectively drops the old categories column
# replace categories with wider ones
app_products["product_category_name_english"] = app_products["product_category_name_english"].apply(replace_cat)
app_products = app_products.rename(columns={'product_category_name_english':'category'})
# one-hot encoding
app_products = pd.get_dummies(app_products, prefix=['category'], columns=['category'])
# new feature
app_products["product_volume_m3"] = app_products["product_length_cm"] * app_products["product_height_cm"] * app_products["product_width_cm"] / 1000000
# drop useless data
for col in ['product_name_lenght', 'category_unknown', "product_length_cm", "product_height_cm", "product_width_cm"]:
    app_products = app_products.drop([col], axis=1)
    
app_products.head()

,product_description_lenght,product_photos_qty,product_weight_g,category_children,category_culture,category_electronics,category_food,category_housecare,category_labor,category_selfcare,category_work,product_volume_m3
product_id,,,,,,,,,,,,
1e9e8ef04dbcff4541ed26657ea517e5,287.0,1.0,225.0,0,0,0,0,0,0,1,0,0.002240
3aa071139cb16b67ca9e5dea641aaa2f,276.0,1.0,1000.0,0,1,0,0,0,0,0,0,0.010800
96bd76ec8810374ed1b65e291975717f,250.0,1.0,154.0,0,1,0,0,0,0,0,0,0.002430
cef67bcfe19066a932b7673e239eb23d,261.0,1.0,371.0,1,0,0,0,0,0,0,0,0.002704
9dc1a7de274444849c219cff195d0b71,402.0,4.0,625.0,0,0,0,0,1,0,0,0,0.004420


In [8]:
app_items = pd.read_csv('./data/olist_order_items_dataset.csv', nrows=num_rows)
app_items = app_items.join(app_products, how='left', on='product_id')

app_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_description_lenght,product_photos_qty,product_weight_g,category_children,category_culture,category_electronics,category_food,category_housecare,category_labor,category_selfcare,category_work,product_volume_m3
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,598.0,4.0,650.0,0,1,0,0,0,0,0,0,0.003528
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,239.0,2.0,30000.0,1,0,0,0,0,0,0,0,0.060000
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,695.0,2.0,3050.0,0,0,0,0,1,0,0,0,0.014157
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,480.0,1.0,200.0,0,0,0,0,0,0,1,0,0.002400
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,409.0,1.0,3750.0,0,0,0,0,0,1,0,0,0.042000


In [9]:
# cleaning
del app_customers, app_orders, app_reviews, app_pay, app_products, app_translate, orders, reviews, payments, cid_to_ucid, oid_to_cuid
gc.collect()

30

# Réduction de dimension

### ACP

In [10]:
def make_acp(list_of_cols):
    # choix du nombre de composantes à calculer
    n_comp = len(list_of_cols)
    # selection des colonnes à prendre en compte dans l'ACP
    data_pca = app[list_of_cols]
    X = data_pca.values
    features = data_pca.columns
    # affichage de la matrice de corrélation
    d = pd.DataFrame(data=app, columns=list_of_cols)
    corr = d.corr()
    mask = np.triu(np.ones_like(corr, dtype=bool))
    f, ax = plt.subplots(figsize=(11, 9))
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.8, center=0,
    square=True, linewidths=.5, cbar_kws={"shrink": .5})
    plt.show()
    # Centrage et Réduction
    std_scale = preprocessing.StandardScaler().fit(X)
    X_scaled = std_scale.transform(X)
    # Calcul des composantes principales
    pca = decomposition.PCA(n_components=n_comp)
    pca.fit(X_scaled)
    # Eboulis des valeurs propres
    display_scree_plot(pca)
    # Cercle des corrélations
    pcs = pca.components_
    display_circles(pcs, n_comp, pca, [(0,1)],
                    labels = np.array(features))
    
    plt.show()
    
    return pcs

In [11]:
# comps = make_acp(app.columns.tolist())

In [12]:
# print(comps)

### TSNE

In [13]:
# app_embedded = TSNE(n_components=2).fit_transform(app)

# Classification non supervisée

In [14]:
# kmeans = KMeans(n_clusters = 6).fit(app)
# kmeans.labels_
# kmeans.cluster_centers_

# Résultats